#  **LLama 3.1 8B com raciocínio (1º Tentativa)**

**Authors:** Renato Freitas Bessa, Adonias Caetano de Oliveira, Ariel Soares Teles

**Institution:** UFDPar - PPGBiotec

**Objective:** responde o exames do revalida com llama 3.1 8B por meio de cadeia de raciocínio.

## **Installation**

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

## **Loading data**

In [ ]:
import pandas as pd
import re
import random

In [ ]:
#sheet_names = ["2017.01", "2017.02", "2020.01", "2020.02", "2021.01", "2022.01", "2022.02", "2023.01", "2023.02", "2024.01"]
sheet_names = ["2024.01"]

#sheet_names = ["2024.01"]

In [ ]:
url = '<public link to access the questions from the 2017.01 to 2024.01 exams>'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
provas = pd.read_excel(read_url)
provas.head()

,Perguntas-2017.01,Gabarito-2017.01,Perguntas-2017.02,Gabarito-2017.02,Perguntas-2020.01,Gabarito-2020.01,Perguntas-2020.02,Gabarito-2020.02,Perguntas-2021.01,Gabarito-2021.01,Perguntas-2022.01,Gabarito-2022.01,Perguntas-2022.02,Gabarito-2022.02,Perguntas-2023.01,Gabarito-2023.01,Perguntas-2023.02,Gabarito-2023.02,Perguntas-2024.01,Gabarito-2024.01
0,1.\tUma mulher com 23 anos de idade comparece ...,NULA,1.\tUma mulher com 38 anos de idade procurou a...,C,1.\tAo atender uma primigesta com 15 anos de i...,A,"1.\tUma mulher com 43 anos de idade, com 80 kg...",C,"1.\tPrimigesta, de 25 anos de idade, com 34 se...",B,"1.\tUma mulher com 51 anos de idade, em tratam...",NULA,1.\tIMAGEM,C,"1.\tUm paciente com 54 anos, em acompanhamento...",D,"1.\tUma mulher de 60 anos, hipertensa, em uso ...",B,"1 Uma mulher com 25 anos, solteira, sem comorb...",A
1,2.\tUm homem com 65 anos de idade encontra-se ...,B,2.\tUma mulher com 23 anos de idade comparece ...,NULA,2.\tUma mulher com 35 anos de idade exerce pro...,C,2.\tAo atender uma primigesta com 15 anos de i...,A,2.\tUm recém-nascido com 20 dias de vida dá en...,C,"2.\tUm paciente com 72 anos de idade, tabagist...",C,"2.\tHomem de 75 anos de idade, tabagista 1 maç...",A,"2.\tUm homem com 62 anos comparece, acompanhad...",D,2.\tUm paciente de 48 anos é internado devido ...,D,"2 Um paciente com 28 anos, vítima de atropelam...",A
2,3.\tUma mulher com 38 anos de idade procurou a...,C,3.\tUm homem com 65 anos de idade encontra-se ...,B,"3.\tUma mulher com 43 anos de idade, com 80 kg...",C,3.\tUma mulher com 35 anos de idade exerce pro...,C,"3.\t Um homem, de 37 anos de idade, com AIDS/H...",B,"3.\tUm menino com 5 anos de idade, acompanhado...",C,3.\tLactente de 6 meses está sendo atendido na...,B,"3.\tUma lactente com 6 meses, nascida a termo,...",B,"3.\tUm menino de 9 anos, com peso de 29 kg e a...",D,3 Um lactente com 3 meses comparece a uma cons...,C
3,4.\tIMAGEM,IMAGEM,"4.\tUma mulher com 32 anos de idade, no quinto...",B,4.\tUma mulher com 18 anos de idade vai ao amb...,A,4.\tUma mulher com 48 anos de idade vai a uma ...,D,4.\tPaciente teve parto cesárea há 8 dias e re...,D,4.\tSegundo o Instituto Nacional do Câncer (IN...,D,"4.\tAdolescente de 14 anos de idade, primípara...",D,"4.\tUma paciente com 21 anos, nuligesta, compa...",A,4.\tUma mulher de 49 anos comparece à unidade ...,B,"4 Uma gestante com 24 anos, G2 P1 A0, chega à ...",D
4,"5.\tUma criança com 4 anos de idade, cujos pai...",A,5.\tIMAGEM,B,5.\tUma criança com 6 anos de idade é levada p...,B,5.\tUma mulher com 18 anos de idade vai ao amb...,A,"5.\tPaciente de 68 anos de idade, do sexo masc...",B,"5.\tUm homem com 48 anos de idade é obeso, tab...",A,5.\tUma mulher de 34 anos com diagnóstico de d...,D,5.\tSindemia é um conjunto de problemas de saú...,A,5.\tEm um território com população em situação...,B,5 Ao assumir a coordenação de uma equipe de sa...,A


## **Unsloth**

**Unsloth models to test**

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 8192,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.5.1+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.11 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((409

## **Respondendo as questões do Revalida**

In [ ]:
respostas_llm = {}

for name in sheet_names:
  perguntas = provas['Perguntas-' + name].to_list()
  gabaritos = provas['Gabarito-' + name].to_list()
  respostas_llm['Repostas-' + name] = []

  for pergunta, gabarito in zip(perguntas, gabaritos):

    if gabarito != "IMAGEM" and gabarito != "TABELA" and gabarito != "NULA":

      prompt = f"Dado a questão a seguir, qual item a, b, c ou d está correto? Pense passo a passo. Questão: " + pergunta

      prompt_tokenizado = tokenizer([prompt], return_tensors='pt').to('cuda')
      FastLanguageModel.for_inference(model)
      output = model.generate(**prompt_tokenizado, max_new_tokens=1024)
      resposta_ids = output[0]
      resposta_texto = tokenizer.decode(resposta_ids, skip_special_tokens=True)

      respostas_llm['Repostas-' + name].append(resposta_texto)
    else:
      respostas_llm['Repostas-' + name].append(gabarito)

In [ ]:
from google.colab import files

df = pd.DataFrame(respostas_llm)
df.to_excel('respostas.xlsx', index=False)

# Especificar o caminho do arquivo no Google Drive
files.download('respostas.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>